# Investigation A (Revisit) & Investigation B (Seasonal LR)

This notebook contains a clean, runnable pipeline that performs:

- Investigation A (Revisit): cleaning, feature engineering, OLS regression, diagnostics.
- Investigation B: merge datasets, seasonal (winter vs spring) LR models, model performance comparison.

Place `dataset1.csv` and `dataset2.csv` in the same folder (`/mnt/data`) and run cells in order. Visuals (distribution, boxplots, residuals, performance plots) are included.


In [ ]:

# Cell 1 — Imports & config
import warnings
warnings.filterwarnings("ignore")

import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import statsmodels.api as sm, statsmodels.formula.api as smf
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import mstats
sns.set(style='whitegrid')

OUT = Path('/mnt/data')
OUT.mkdir(exist_ok=True)
print('Outputs will be saved to', OUT)

# User settings
FORCE_RESPONSE = None       # set column name to force response for Investigation A
WINSORIZE = True
WINSOR_PCT = 0.01
MIN_ROWS_SEASON = 20


In [ ]:

# Cell 2 — Load dataset1 and dataset2 (ensure files are in /mnt/data)
d1 = OUT / 'dataset1.csv'
d2 = OUT / 'dataset2.csv'
assert d1.exists(), 'dataset1.csv not found in /mnt/data'
assert d2.exists(), 'dataset2.csv not found in /mnt/data'
df1 = pd.read_csv(d1)
df2 = pd.read_csv(d2)
# standardize column names
df1.columns = df1.columns.str.strip().str.lower().str.replace(' ', '_')
df2.columns = df2.columns.str.strip().str.lower().str.replace(' ', '_')
print('df1 shape:', df1.shape, 'df2 shape:', df2.shape)
df1.head(2)


In [ ]:

# Cell 3 — Cleaning & basic feature engineering
# Numeric casting
nums = ['bat_landing_to_food','seconds_after_rat_arrival','hours_after_sunset','rat_minutes','rat_arrival_number','food_availability','bat_landing_number']
for c in nums:
    if c in df1.columns: df1[c] = pd.to_numeric(df1[c], errors='coerce')
    if c in df2.columns: df2[c] = pd.to_numeric(df2[c], errors='coerce')

# Datetime parsing if present
for col in ['start_time','rat_period_start','rat_period_end','sunset_time']:
    if col in df1.columns:
        df1[col] = pd.to_datetime(df1[col], errors='coerce', dayfirst=True)
if 'time' in df2.columns:
    df2['time'] = pd.to_datetime(df2['time'], errors='coerce', dayfirst=True)

# rat_present for df1
df1['rat_present'] = np.nan
if 'seconds_after_rat_arrival' in df1.columns:
    df1.loc[df1['seconds_after_rat_arrival'].notna(), 'rat_present'] = (df1.loc[df1['seconds_after_rat_arrival'].notna(), 'seconds_after_rat_arrival'] >= 0).astype(int)
if set(['start_time','rat_period_start','rat_period_end']).issubset(df1.columns):
    mask = df1['rat_present'].isna() & df1['start_time'].notna() & df1['rat_period_start'].notna() & df1['rat_period_end'].notna()
    df1.loc[mask, 'rat_present'] = ((df1.loc[mask,'start_time'] >= df1.loc[mask,'rat_period_start']) & (df1.loc[mask,'start_time'] <= df1.loc[mask,'rat_period_end'])).astype(int)
df1['rat_present'] = df1['rat_present'].fillna(0).astype(int)

# vigilance_time
if 'bat_landing_to_food' in df1.columns:
    df1['vigilance_time'] = df1['bat_landing_to_food'].astype(float)

# season_label mapping
if 'season' in df1.columns:
    df1['season'] = pd.to_numeric(df1['season'], errors='coerce')
    df1['season_label'] = df1['season'].map({0:'winter',1:'spring'})
else:
    df1['season_label'] = df1.get('season_label', pd.NA)

# rat_activity_index in df2
if set(['rat_arrival_number','rat_minutes']).issubset(df2.columns):
    df2['rat_activity_index'] = df2['rat_arrival_number'].fillna(0) * df2['rat_minutes'].fillna(0)
elif 'rat_arrival_number' in df2.columns:
    df2['rat_activity_index'] = df2['rat_arrival_number'].fillna(0)
elif 'rat_minutes' in df2.columns:
    df2['rat_activity_index'] = df2['rat_minutes'].fillna(0)
else:
    df2['rat_activity_index'] = 0

print('Cleaning and basic feature engineering done.')
df1.head(2)


In [ ]:

# Cell 4 — Select response variable and EDA for Investigation A
# Choose response: either forced or auto-select based on presence of bat-related columns
if FORCE_RESPONSE and FORCE_RESPONSE in df1.columns:
    response = FORCE_RESPONSE
else:
    candidates = [c for c in df1.select_dtypes(include=[np.number]).columns if c not in ['month','season','rat_present','hours_after_sunset']]
    response = 'bat_landing_to_food' if 'bat_landing_to_food' in candidates else (candidates[0] if candidates else 'vigilance_time')
print('Investigation A response chosen:', response)

# Basic descriptives
print('Descriptive stats (response):\n', df1[response].describe())
print('\nBy rat_present:')
print(df1.groupby('rat_present')[response].agg(['count','mean','median','std']))

# Plots
plt.figure(figsize=(6,3))
sns.histplot(df1[response].dropna(), bins=40, kde=True)
plt.title(f'Distribution of {response}'); plt.tight_layout(); plt.savefig(OUT/f'{response}_dist.png'); plt.close()

plt.figure(figsize=(6,3))
sns.boxplot(data=df1, x='rat_present', y=response, showmeans=True)
plt.title(f'{response} by rat_present'); plt.tight_layout(); plt.savefig(OUT/f'{response}_by_rat_present.png'); plt.close()
print('Saved EDA plots.')


In [ ]:

# Cell 5 — Investigation A: OLS regression + ANOVA
predictorsA = [p for p in ['seconds_after_rat_arrival','rat_present','risk','reward','hours_after_sunset'] if p in df1.columns]
modA = df1.dropna(subset=[response] + predictorsA).copy()
terms = []
for c in predictorsA:
    if c in ['risk','reward','rat_present']:
        terms.append(f'C({c})')
    else:
        terms.append(c)
formulaA = response + ' ~ ' + ' + '.join(terms) if terms else response + ' ~ 1'
print('Formula A:', formulaA)
modelA = smf.ols(formulaA, data=modA).fit()
print(modelA.summary())
sm.stats.anova_lm(modelA, typ=2).to_csv(OUT/'investigationA_anova_table.csv')
with open(OUT/'investigationA_ols_summary.txt','w') as f: f.write(modelA.summary().as_text())
# Diagnostics plots
resid = modelA.resid; fitted = modelA.fittedvalues
plt.figure(figsize=(6,3)); plt.scatter(fitted, resid, alpha=0.5); plt.axhline(0, color='r', linestyle='--'); plt.title('A: Residuals vs Fitted'); plt.tight_layout(); plt.savefig(OUT/'A_resid_vs_fitted.png'); plt.close()
sm.qqplot(resid, line='45'); plt.title('A: QQ'); plt.tight_layout(); plt.savefig(OUT/'A_qq.png'); plt.close()
print('Investigation A outputs saved.')


In [ ]:

# Cell 6 — Merge df1 and df2 for Investigation B
merged = df1.copy()
if set(['hours_after_sunset','month']).issubset(df1.columns) and set(['hours_after_sunset','month']).issubset(df2.columns):
    m1 = df1.sort_values('hours_after_sunset').reset_index(drop=True)
    m2 = df2.sort_values('hours_after_sunset').reset_index(drop=True)
    try:
        merged = pd.merge_asof(m1, m2, on='hours_after_sunset', by='month', direction='nearest', tolerance=0.5, suffixes=('','_sess'))
        print('Merged shape:', merged.shape)
    except Exception as e:
        print('merge_asof failed:', e)
        merged = df1.copy()
else:
    print('Merge not possible; using df1 copy.')

merged.to_csv(OUT/'merged_df_for_investigationB.csv', index=False)
merged.head(2)


In [ ]:

# Cell 7 — Prepare merged dataset: winsorize, log-transform, interactions, scaling
data = merged.copy()
response = 'bat_landing_to_food' if 'bat_landing_to_food' in data.columns else 'vigilance_time'
data = data[data[response].notna()].copy()
data = data[data[response] >= 0]

if WINSORIZE:
    data[response + '_winsor'] = mstats.winsorize(data[response].fillna(0), limits=(WINSOR_PCT, WINSOR_PCT))
else:
    data[response + '_winsor'] = data[response].fillna(0)

data['y_log'] = np.log1p(data[response + '_winsor'])

cont_candidates = ['rat_activity_index','rat_minutes','rat_arrival_number','bat_landing_number','food_availability','hours_after_sunset','seconds_after_rat_arrival']
cat_candidates = ['rat_present','risk','reward','habit']

predictors = [c for c in cont_candidates + cat_candidates if c in data.columns]

if 'rat_activity_index' in data.columns and 'rat_present' in data.columns:
    data['ratact_x_ratpresent'] = data['rat_activity_index'] * data['rat_present']
    predictors.append('ratact_x_ratpresent')

if 'food_availability' in data.columns and 'season' in data.columns:
    data['season_num'] = data['season'].map({0:0,1:1}) if 'season' in data.columns else data['season_label'].map({'winter':0,'spring':1})
    data['food_x_season'] = data['food_availability'] * data['season_num']
    predictors.append('food_x_season')

numeric_preds = [p for p in predictors if p in cont_candidates + ['ratact_x_ratpresent','food_x_season','seconds_after_rat_arrival']]
scaler = StandardScaler()
if numeric_preds:
    data[numeric_preds] = scaler.fit_transform(data[numeric_preds].fillna(0))

data.to_csv(OUT/'merged_scaled_for_modeling.csv', index=False)
print('Prepared merged dataset. predictors:', predictors)


In [ ]:

# Cell 8 — Backward selection by AIC on y_log
def backward_aic(df, response_col, predictors_list, categorical_prefix=None):
    best_preds = predictors_list.copy()
    categorical_prefix = categorical_prefix or []
    def formula(preds):
        terms = []
        for p in preds:
            if p in categorical_prefix:
                terms.append(f"C({p})")
            else:
                terms.append(p)
        return response_col + ' ~ ' + ' + '.join(terms) if terms else response_col + ' ~ 1'
    current_model = smf.ols(formula(best_preds), data=df).fit()
    current_aic = current_model.aic
    improved = True
    while improved and len(best_preds) > 0:
        improved = False
        candidates = []
        for p in best_preds:
            trial = [x for x in best_preds if x != p]
            try:
                m = smf.ols(formula(trial), data=df).fit()
                candidates.append((p, m.aic))
            except:
                candidates.append((p, np.inf))
        p_remove, best_aic = min(candidates, key=lambda x: x[1])
        if best_aic + 1e-6 < current_aic:
            best_preds.remove(p_remove)
            current_aic = best_aic
            improved = True
    return smf.ols(formula(best_preds), data=df).fit(), best_preds

categorical_vars = [c for c in ['rat_present','risk','reward','habit'] if c in data.columns]
initial_preds = [p for p in predictors if p in data.columns]
print('Initial preds:', initial_preds)
model_aic, selected_preds = backward_aic(data.dropna(subset=['y_log']), 'y_log', initial_preds, categorical_prefix=categorical_vars)
print('Selected preds:', selected_preds)
with open(OUT/'investigationB_selected_model_summary.txt','w') as f: f.write(model_aic.summary().as_text())


In [ ]:

# Cell 9 — Final model diagnostics & performance (full)
final_model = model_aic
print(final_model.summary())

resid = final_model.resid; fitted = final_model.fittedvalues
plt.figure(figsize=(6,3)); plt.scatter(fitted, resid, alpha=0.5); plt.axhline(0,color='r',linestyle='--'); plt.title('Final model resid vs fitted'); plt.tight_layout(); plt.savefig(OUT/'final_resid_vs_fitted.png'); plt.close()
sm.qqplot(resid, line='45'); plt.title('Final model QQ'); plt.tight_layout(); plt.savefig(OUT/'final_qq.png'); plt.close()

# VIF
try:
    exog = final_model.model.exog
    vif = pd.DataFrame({'variable': final_model.model.exog_names, 'VIF':[variance_inflation_factor(exog, i) for i in range(exog.shape[1])]})
    vif.to_csv(OUT/'final_vif.csv', index=False)
except Exception as e:
    print('VIF failed:', e)

# Performance metrics (safe)
y_true_log = data.loc[final_model.model.data.row_labels, 'y_log']
y_pred_log = final_model.fittedvalues
mask = np.isfinite(y_true_log) & np.isfinite(y_pred_log)
if mask.sum() > 0:
    rmse_log = np.sqrt(mean_squared_error(y_true_log[mask], y_pred_log[mask]))
    r2_log = final_model.rsquared
    rmse_orig = np.sqrt(mean_squared_error(np.expm1(y_true_log[mask]), np.expm1(y_pred_log[mask])))
    mae_orig = mean_absolute_error(np.expm1(y_true_log[mask]), np.expm1(y_pred_log[mask]))
else:
    rmse_log = r2_log = rmse_orig = mae_orig = np.nan

pd.DataFrame({'metric':['rmse_log','r2_log','rmse_orig','mae_orig'],'value':[rmse_log,r2_log,rmse_orig,mae_orig]}).to_csv(OUT/'final_model_performance.csv', index=False)
pd.DataFrame({'coef': final_model.params}).to_csv(OUT/'final_model_coeffs.csv')
print('Final model artifacts saved.')


In [ ]:

# Cell 10 — Seasonal models and performance comparison
data['season_label'] = data['season_label'].astype(str)
winter_df = data[data['season_label']=='winter'].copy()
spring_df = data[data['season_label']=='spring'].copy()
print('Winter n=', winter_df.shape[0], 'Spring n=', spring_df.shape[0])

season_models = {}
for name, df_s in [('winter', winter_df), ('spring', spring_df)]:
    if df_s.shape[0] >= MIN_ROWS_SEASON:
        terms = [f'C({p})' if p in categorical_vars else p for p in selected_preds]
        formula_s = 'y_log ~ ' + ' + '.join(terms) if terms else 'y_log ~ 1'
        try:
            m = smf.ols(formula_s, data=df_s).fit()
            season_models[name] = m
            with open(OUT/f'investigationB_ols_{name}_summary.txt','w') as f: f.write(m.summary().as_text())
            print(name, 'model R2_log=', m.rsquared)
        except Exception as e:
            print('Fit failed for', name, e)
    else:
        print('Not enough rows for', name)

# Performance comparison (safe NaN handling)
perf_rows = []
for name, df_s in [('full', data), ('winter', winter_df), ('spring', spring_df)]:
    if name == 'full':
        m = final_model
        df_use = data.dropna(subset=['y_log'])
    else:
        m = season_models.get(name)
        df_use = df_s.dropna(subset=['y_log'])
    if m is None or df_use.empty:
        perf_rows.append({'Model':name,'R2_log':np.nan,'RMSE_log':np.nan,'RMSE_orig':np.nan,'NumObs':int(df_use.shape[0])}); continue
    y_true_log = df_use['y_log']
    y_pred_log = m.predict(df_use)
    mask = np.isfinite(y_true_log) & np.isfinite(y_pred_log)
    if mask.sum() == 0:
        perf_rows.append({'Model':name,'R2_log':np.nan,'RMSE_log':np.nan,'RMSE_orig':np.nan,'NumObs':int(df_use.shape[0])}); continue
    rmse_log = np.sqrt(mean_squared_error(y_true_log[mask], y_pred_log[mask]))
    rmse_orig = np.sqrt(mean_squared_error(np.expm1(y_true_log[mask]), np.expm1(y_pred_log[mask])))
    perf_rows.append({'Model':name,'R2_log':float(m.rsquared),'RMSE_log':float(rmse_log),'RMSE_orig':float(rmse_orig),'NumObs':int(df_use.shape[0])})

perf_df = pd.DataFrame(perf_rows)
perf_df.to_csv(OUT/'seasonal_model_performance_comparison.csv', index=False)
print(perf_df)

# Plot comparison
plt.figure(figsize=(7,4))
perf_df.set_index('Model')[['R2_log','RMSE_log']].plot(kind='bar', secondary_y='RMSE_log', rot=0)
plt.title('Seasonal Model Performance (R2_log and RMSE_log)'); plt.tight_layout(); plt.savefig(OUT/'seasonal_model_performance_plot.png'); plt.close()
print('Seasonal performance plot saved.')


## Wrap-up

Key outputs saved to `/mnt/data` (or local folder where notebook runs):

- investigationA_ols_summary.txt
- investigationA_anova_table.csv
- final_model_coeffs.csv, final_model_performance.csv
- investigationB_ols_winter_summary.txt, investigationB_ols_spring_summary.txt (if generated)
- seasonal_model_performance_comparison.csv, seasonal_model_performance_plot.png

Run the notebook end-to-end in Jupyter. If column names differ, update predictor lists accordingly.